In [1]:
# Import and intizialize
import pandas as pd
import json
from math import isnan


In [2]:
# Read datasets
migration_flow = pd.read_csv('migration_flows_bilateral_2015_clean.csv')
iso_codes = pd.read_csv('all.csv')

In [3]:
# Copy for splitting csv into two without changing the original variable
immigrants = migration_flow.copy()
emigrants = migration_flow.copy()

# Immigrants
immigrants = immigrants[immigrants.columns.drop(list(immigrants.filter(regex='EMIGRANTS')))]
immigrants = immigrants.iloc[:,:-8]

# Emigrants
emigrants = emigrants[emigrants.columns.drop(list(emigrants.filter(regex='IMMIGRANTS')))]
emigrants = emigrants.iloc[:,:-8] # drop last 8 columns since they are micro nations
emigrants.columns = emigrants.columns.str.replace("EMIGRANTS_FROM_","")

In [4]:
# Create a dictionary to rename column names
countries = emigrants.columns[2:] 

# Create a dictionary with country names as key and index as value
country_dict = {key: json.dumps(idx) for idx, key in enumerate(countries)}

# Replace column headers with numbers instead of country names
emigrants.rename(columns=country_dict, inplace=True) 

# convert dataframe to dictionary
flow = emigrants.iloc[:,2:].to_dict(orient='index')

# remove all nan values so dictionary is clean 
for i in flow: 
    for j in list(flow[i]):
        if isnan(flow[i][j]):
            del flow[i][j]
flower = json.dumps(flow)

In [5]:
# Add the flow dictionary values to column in dataframe where the key is row number
emigrants_15 = emigrants.iloc[:,1:2]
emigrants_15['flows'] = pd.Series(flow)
emigrants_15['short'] = emigrants_15['COUNTRY'].str[:4]

C:\Users\silva\AppData\Local\Temp\ipykernel_26940\3441384611.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  emigrants_15['flows'] = pd.Series(flow)
C:\Users\silva\AppData\Local\Temp\ipykernel_26940\3441384611.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  emigrants_15['short'] = emigrants_15['COUNTRY'].str[:4]


In [6]:
# Rearrange iso dataset for join with migration dataset
iso_codes['name'] = iso_codes['name'].str.upper()
iso = iso_codes.iloc[:,:2]
iso = iso.rename(columns = {'name':'COUNTRY', 'alpha-2':'ISO Code'})
iso['short'] = iso['COUNTRY'].str[:4]
iso = iso.iloc[:,1:3]

# Actual join via merge()
iso_emigrants = pd.merge(emigrants_15, iso, how="left", on=['short'])

In [7]:
# Manual data cleaning -> future check if values are correctly assigned!

# Cleaning ISO values
iso_emigrants.xs(39)['ISO Code'] = 'CV'
iso_emigrants.xs(40)['ISO Code'] = 'MQ' # As of now Martinique 
iso_emigrants.xs(44)['ISO Code'] = 'JE' # As of now Jersey
iso_emigrants.xs(53)['ISO Code'] = 'CI'
iso_emigrants.xs(59)['ISO Code'] = 'CD'
iso_emigrants.xs(74)['ISO Code'] = 'FO'
iso_emigrants.xs(125)['ISO Code'] = 'LA'
iso_emigrants.xs(162)['ISO Code'] = 'NA'
iso_emigrants.xs(193)['ISO Code'] = 'PF' # As of now French Polynesia
iso_emigrants.xs(197)['ISO Code'] = 'RE'
iso_emigrants.xs(323)['ISO Code'] = 'VA'

# We do not have values for North/South Korea



In [8]:
# Create a csv file to join in arcGis
iso_emigrants.to_csv('emi_15_flow.csv', index=False, encoding='utf-8')

In [3]:
# Import and intizialize
import pandas as pd
import json
from math import isnan

In [10]:
# Reintroduce the argis geoJson for further processing
f = open('i20.geoJson', 'r')
data = json.load(f)
f.close()

In [11]:
for i in range(len(data['features'])):
    string = data['features'][i]['properties']['flows']
    string = string.replace("'",'"') 
    data['features'][i]['properties']['flows'] = json.loads(string)

In [12]:
# Change flows to json format instead of string
for i in range(len(data['features'])):
    flow_json = iso_emigrants['flows'][i]
    data['features'][i]['properties']['flows'] = json.dumps(flow_json)

for i in range(len(data['features'])):
    flow_json = json.loads(data['features'][i]['properties']['flows'])
    data['features'][i]['properties']['flows'] = flow_json

In [13]:
# Create a centroid attribute where x,y are elements in list centroid 
for i in data['features']:
    i['geometry']['centroid'] = [i['properties']['X'],i['properties']['Y']]

# X is long, y is lat in dataset from arcgis

In [14]:
data['features'][5]['properties']

{'OBJECTID': 7,
 'Country': '6',
 'ISO Code': 'WS',
 'Affiliation': 'Samoa',
 'Affiliation ISO Code': 'WS',
 'X': -172.1594594644777,
 'Y': -13.758366525659419,
 'Shape_Length': 3.0196618946939964,
 'Shape_Area': 0.24551890561299944,
 'ID': 6,
 'COUNTRY': 'SAMOA',
 'flows': {'5': -618288.0,
  '37': 8662.0,
  '38': 0,
  '41': 31.0,
  '47': 997.0,
  '56': 75667.0,
  '88': 9252.0,
  '141': 235.0,
  '143': 839.0,
  '167': 0,
  '168': 5867.0,
  '174': 1186.0,
  '184': 6441.0,
  '196': 5917.0,
  '235': 290.0},
 'short': 'SAMOA'}

In [2]:
# Write Json File
json_object = json.dumps(data)
with open('i20.json', 'w') as outfile:
    outfile.write(json_object)


NameError: name 'json' is not defined

Part ot calculate the difference in migration between the years:

In [4]:
# Reintroduce the argis geoJson for further processing
f = open('i20.json', 'r')
i20 = json.load(f)
f.close()

f = open('i15.json', 'r')
i15 = json.load(f)
f.close()

f = open('i10.json', 'r')
i10 = json.load(f)
f.close()

f = open('e20.json', 'r')
e20 = json.load(f)
f.close()

f = open('e15.json', 'r')
e15 = json.load(f)
f.close()

f = open('e10.json', 'r')
e10 = json.load(f)
f.close()

In [42]:
## Iterate through flows of every country and then compare it key in order to match & subtract

e15_20 = []

i20['features'][i]['properties']['flows']

counter = 0

for i in range(len(e20['features'])):
        for key in e15['features'][i]['properties']['flows']:
            k = e20['features'][i]['properties']['flows'][key]
            l = e15['features'][i]['properties']['flows'][key]
            value = k-l
            e15_20.append({key: value})



In [44]:
e15_20 = []

i20['features'][0]['properties']['flows']

counter = 0

for key in e15['features'][0]['properties']['flows']:
    k = e20['features'][0]['properties']['flows'][key]
    l = e15['features'][0]['properties']['flows'][key]
    value = k-l
    e15_20.append({key: value})

In [50]:
len(e10['features'][0]['properties']['flows'])

11

In [45]:
e15_20[1]

{'37': 0}

In [20]:
len(i20['features'][0]['properties']['flows'])

6